# Chapter 1 - Introduction / Business Problem 

## Housing shortage in the Netherlands
Young people in the Netherlands are strugling to buy their first home. There is a limited supply available and investers buy a lot of it to convert to cheap rentals. Thus leaving the first time buyers with little chance of actually obtaining there first home. It's really frustrating for this group and the government is trying to help them out, with little succes so far. 

We see a lot of these first time buyers looking for a house in a certain area and if they are unable to actually buy something the will broaden there search area. The analyses in this report will help these first time buyers, and everyone else looking for a new home to broaden there horizon. Find different areas where housing prices may be lower, and the markter less overheated while still having more or less the same services available. This will dramatically increase there chance of finding a suitable home and making a good personal investment. 

### Densely populated country
The Netherlands is a densely populated country with over 412.49 people per sqaure kilometer (p/km2). As a comparison the USA has a population density of 33.67 p/km2. Earning a place in the top 20 of most densely populated countries. 

https://www.populationpyramid.net/population-density/2020/

### Growing population
The population in the Netherlands is still growing. Allthough the rate at which it is increasing is getting slower. Every year there is a need for more houses, conflicting with other interests like nature, agriculture, office space, airfields, windmills, etc. 

https://www.worldometers.info/world-population/netherlands-population/

### Decreasing household size 
There is a clearly visable trend in the size of the average household size (number of people in a household) in the last decades. The household size is shrinking steadily from 1970 until present day. More and more single person households or couples without children are buying or renting houses. The number of households keeps on increasing even if the total population would stay the same. The increase in households means we need more homes for the same number of people. 

https://themasites.pbl.nl/balansvandeleefomgeving/jaargang-2014/wonen-en-vastgoed/passendheid-conclusies/passendheid-huishoudens

### Is the whole of the Netherlands overpopulated? 
On general the population of the Netherlands is increasing but there are a lot of regional exeptions. There are areas of the Netherlands where the population is shrinking. These places would welcome the influx of new people! Because who wants to live in a ghost town? In the period 2015-2020 the area's near the borders are the primary areas with a decrease in population density.

https://www.cbs.nl/nl-nl/dossier/dossier-verstedelijking/hoofdcategorieen/waar-groeit-of-krimpt-de-bevolking-

## Research questions: Where the hell can I buy a house in the Netherlands? 

#### Why are some area's more atractive then others? 
#### What are the differences and similarieties between the municipallities (or clusters)?
#### Is there a correlation between the housing price and population growth?
#### Can we predict population growth based on location data?
#### How accurate can we model population growth/shirinkage based on location data? 
#### What is causing population growth of shrinkage?  
#### Can we reverse depopulation by stimulating certain venues or services?


# Chapter 2- The Data
In this research I will be combining data from two different sources to be able to answer the research questions. 

## Part one -  Central bureau of statistics (CBS) 
From the CBS we will get data about the municipalities, including population growth and density, avarage house prices, coordinates, etc. This will be the starting point for us in building a data frame to answer our research questions. This data is publically available on https://www.cbs.nl/. It is all structered tabular data in CSV file. To be able to use the data I will combine differend data sets and clean the data to be able to properly work with it. 

## Part two - Foursquare 
Foursquare will provide us with additional data about available venues in these municipalities. Using the Foursquare API I will gather information about (popular) venues in the municipalities. Based on the most popular venues the municipalities will be clustered together using a KNN algorithm. We will add the location data and clusters to the municipality dataframe. 

# Cleaning the data

## Part one - CBS 

### Municipallity dataframe from CBS data
To start with will load, clean en merge several CSV files into one dataframe. This will be the backbone for our further analyses. 

In [94]:
## Importing libraries

import numpy as np # library to handle data in a vectorized manner

!conda install pandas
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# imports needed to acces data from IBM cloud storage
import os, types
from botocore.client import Config
import ibm_boto3

import json # library to handle JSON files
from pandas import json_normalize # tranform JSON file into a pandas dataframe
import requests # to communicate with API

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans


# map rendering library folium
# !conda install -c conda-forge Folium=0.5.0 --yes 
import folium 

print('libraries imported succesfully')

Solving environment: done

# All requested packages already installed.

libraries imported succesfully


In [95]:
# The code was removed by Watson Studio for sharing.

In [99]:
# The code was removed by Watson Studio for sharing.

AttributeError: 'NoneType' object has no attribute 'items'

            Naam      Type Provincie afkorting Bezoekadres straat  \
0    Aa en Hunze  Gemeente                  DR       Spiekersteeg   
1       Aalsmeer  Gemeente                  NH      Raadhuisplein   
2         Aalten  Gemeente                  GD          Hofstraat   
3  Achtkarspelen  Gemeente                  FR     Stationsstraat   
4   Alblasserdam  Gemeente                  ZH           Cortgene   

  Bezoekadres huisnummer  Bezoekadres toevoeging Bezoekadres postcode  \
0                      1                     NaN              9461 BH   
1                      1                     NaN              1431 EH   
2                      8                     NaN              7121 DM   
3                     18                     NaN              9285 NH   
4                      2                     NaN              2951 ED   

  Bezoekadres plaats  Bezoekadres latitude  Bezoekadres longitude  \
0             GIETEN                53.005                  6.760   
1       

In [84]:
df_CBS = df_data_1
df_data_1.head()

AttributeError: 'NoneType' object has no attribute 'items'

            Naam      Type Provincie afkorting Bezoekadres straat  \
0    Aa en Hunze  Gemeente                  DR       Spiekersteeg   
1       Aalsmeer  Gemeente                  NH      Raadhuisplein   
2         Aalten  Gemeente                  GD          Hofstraat   
3  Achtkarspelen  Gemeente                  FR     Stationsstraat   
4   Alblasserdam  Gemeente                  ZH           Cortgene   

  Bezoekadres huisnummer  Bezoekadres toevoeging Bezoekadres postcode  \
0                      1                     NaN              9461 BH   
1                      1                     NaN              1431 EH   
2                      8                     NaN              7121 DM   
3                     18                     NaN              9285 NH   
4                      2                     NaN              2951 ED   

  Bezoekadres plaats  Bezoekadres latitude  Bezoekadres longitude  \
0             GIETEN                53.005                  6.760   
1       

In [80]:
## Cleaning the dataframe
df_mun = df_CBS[['Naam', 'Bezoekadres latitude', 'Bezoekadres longitude', "Oppervlakte (km2)", 
        'Aantal inwoners', "Inwoners per km2"]] 

df_mun = df_mun.rename(columns={"Naam":"municipallity", "Bezoekadres latitude":"latitude", 
                       'Bezoekadres longitude':'longitude', "Oppervlakte (km2)": "surface area (km2)", 
                      'Aantal inwoners':"population", "Inwoners per km2" : "population density per km2"})

df_mun.info() # first check on the dataframe we can see some municipallities have no location data
df_mun.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 352 entries, 0 to 351
Data columns (total 6 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   municipallity               352 non-null    object 
 1   latitude                    344 non-null    float64
 2   longitude                   344 non-null    float64
 3   surface area (km2)          352 non-null    int64  
 4   population                  352 non-null    int64  
 5   population density per km2  352 non-null    int64  
dtypes: float64(2), int64(3), object(1)
memory usage: 16.6+ KB


AttributeError: 'NoneType' object has no attribute 'items'

   municipallity  latitude  longitude  surface area (km2)  population  \
0    Aa en Hunze    53.005      6.760                 279       25390   
1       Aalsmeer    52.267      4.750                  34       31499   
2         Aalten    51.926      6.583                  97       26962   
3  Achtkarspelen    53.255      6.143                 104       27935   
4   Alblasserdam    51.862      4.658                  10       20014   

   population density per km2  
0                          90  
1                         928  
2                         277  
3                         268  
4                        2001  

In [6]:
# drop municipallities without coordinates 
df_mun = df_mun.dropna() 
df_mun.describe() # Shows there is at least one municipallity with a surface area of 0 km 

AttributeError: 'NoneType' object has no attribute 'items'

         latitude   longitude  surface area (km2)     population  \
count  344.000000  344.000000          344.000000     344.000000   
mean    52.048712    5.394965          103.970930   48316.843023   
std      0.560028    0.766609          107.316943   72488.068116   
min     50.771000    3.487000            0.000000     932.000000   
25%     51.647250    4.799500           33.000000   21252.750000   
50%     52.026000    5.410000           73.000000   29821.500000   
75%     52.379000    5.965750          124.500000   48346.500000   
max     53.479000    7.071000          765.000000  854047.000000   

       population density per km2  
count                3.440000e+02  
mean                 4.168147e+04  
std                  5.451017e+05  
min                  3.000000e+00  
25%                  2.267500e+02  
50%                  4.385000e+02  
75%                  1.099500e+03  
max                  8.458800e+06  

In [7]:
df_mun.loc[df_mun["surface area (km2)"] == 0] 
## there are two municipallities with a surface area of 0 and a very large population density as a result
## in order to be able to proceed our analyses we will drop these rows 


AttributeError: 'NoneType' object has no attribute 'items'

      municipallity  latitude  longitude  surface area (km2)  population  \
159  Krimpenerwaard    51.973      4.771                   0       55644   
204      Nissewaard    51.848      4.329                   0       84588   

     population density per km2  
159                     5564400  
204                     8458800  

In [9]:
df_mun.drop(index = [159, 204], inplace=True)

In [10]:
df_mun.reset_index(drop=True, inplace=True)
df_mun.describe()

AttributeError: 'NoneType' object has no attribute 'items'

         latitude   longitude  surface area (km2)     population  \
count  342.000000  342.000000          342.000000     342.000000   
mean    52.049520    5.399906          104.578947   48189.362573   
std      0.561548    0.765923          107.334533   72672.598162   
min     50.771000    3.487000            1.000000     932.000000   
25%     51.643750    4.801500           34.000000   21226.250000   
50%     52.030500    5.416500           73.000000   29754.000000   
75%     52.383000    5.967250          125.500000   47714.000000   
max     53.479000    7.071000          765.000000  854047.000000   

       population density per km2  
count                  342.000000  
mean                   921.710526  
std                   1803.437871  
min                      3.000000  
25%                    226.250000  
50%                    436.500000  
75%                   1085.500000  
max                  28637.000000  

In [73]:
from math import sqrt #import the function to calculate the sqaure root

for ind, row in df_mun.iterrows():
    df_mun.loc[ind, 'approximate radius (m)'] = sqrt(row['surface area (km2)'] / 3.142) * 1000 ## We approximated the radius of the municipallity by acting as if it are all perfect circles. In this way we will more or less get the average radius.
df_mun['approximate radius (m)'] = df_mun['approximate radius (m)'].astype(int)
    
df_mun.head()

,municipallity,latitude,longitude,surface area (km2),population,population density per km2,approximate radius (m)
0,Aa en Hunze,53.005,6.760,279,25390,90,9423
1,Aalsmeer,52.267,4.750,34,31499,928,3289
2,Aalten,51.926,6.583,97,26962,277,5556
3,Achtkarspelen,53.255,6.143,104,27935,268,5753
4,Alblasserdam,51.862,4.658,10,20014,2001,1784


### importing average house prices
Also from the CBS but in a different dataframe

In [54]:

body = client_6d61fec0a3dd4c5a8549fe3300270ebc.get_object(Bucket='datasciencecapstoneprojectbattleo-donotdelete-pr-wcot4qbo8rnhhg',Key='gemiddelde huizenprijzen.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_house_price = pd.read_csv(body, sep = ';')
df_house_price.head()


AttributeError: 'NoneType' object has no attribute 'items'

     Gemeente Gemiddelde verkoopprijs ( duizend euro)  house price (euro)
0  Appingedam                                   194,8              194800
1    Delfzijl                                   155,1              155100
2  Groningen                                    258,9              258900
3   Loppersum                                   192,4              192400
4      Almere                                   294,3              294300

In [42]:
df_house_price = df_house_price.rename(columns={"Gemeente":"municipallity", "Gemiddelde verkoopprijs ( duizend euro)":
                                                "average house price (1000 euro)"})
df_house_price = df_house_price[['municipallity', "house price (euro)"]]
df_house_price.head()

NameError: name 'df_house_price' is not defined

In [64]:
df_cbs = pd.merge(df_mun, df_house_price)
df_cbs.describe()

,latitude,longitude,surface area (km2),population,population density per km2,approximate radius (m),house price (euro)
count,333.000000,333.000000,333.000000,333.000000,333.000000,333.000000,333.000000
mean,52.049964,5.396339,105.153153,46191.873874,906.204204,5207.768856,310388.588589
std,0.556063,0.768732,108.436912,67541.780748,1799.780005,2522.936204,85320.174747
min,50.771000,3.487000,1.000000,932.000000,3.000000,564.153010,161300.000000
25%,51.649000,4.801000,33.000000,21266.000000,226.000000,3240.812308,258000.000000
50%,52.033000,5.405000,73.000000,29753.000000,432.000000,4820.125431,293700.000000
75%,52.377000,5.968000,127.000000,47537.000000,1075.000000,6357.681541,340900.000000
max,53.479000,7.071000,765.000000,854047.000000,28637.000000,15603.701270,831600.000000


In [65]:
df_cbs.head(10)

,municipallity,latitude,longitude,surface area (km2),population,population density per km2,approximate radius (m),house price (euro)
0,Aa en Hunze,53.005,6.760,279,25390,90,9423.213073,284800
1,Aalsmeer,52.267,4.750,34,31499,928,3289.549063,409300
2,Aalten,51.926,6.583,97,26962,277,5556.262774,226100
3,Achtkarspelen,53.255,6.143,104,27935,268,5753.254413,216500
4,Alblasserdam,51.862,4.658,10,20014,2001,1784.008461,268400
5,Albrandswaard,51.861,4.394,24,25218,1050,2763.774023,361900
6,Alkmaar,52.635,4.746,31,108470,3474,3141.071024,286900
7,Almelo,52.357,6.659,69,72629,1046,4686.206855,210000
8,Almere,52.372,5.222,249,203990,819,8902.184342,294300
9,Alphen aan den Rijn,52.129,4.660,58,109682,1901,4296.461321,286500


### Population growth
Below we are going to add the population growth or shrinkage in the periode of 2015 - 2020 to our dataframe. This is the last step in completing the CBS data.

In [66]:

body = client_6d61fec0a3dd4c5a8549fe3300270ebc.get_object(Bucket='datasciencecapstoneprojectbattleo-donotdelete-pr-wcot4qbo8rnhhg',Key='population growth.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_pop_growth = pd.read_csv(body, sep = ';')

In [67]:
df_pop_growth = df_pop_growth.rename(columns={"Gemeente":"municipallity", "relatieve ontwikkeling 2015-2020 (%)":
                                                "pop"})
df_pop_growth ['population growth (%)'] = df_pop_growth['pop'].str.replace(',','.').astype(float)
df_pop_growth = df_pop_growth [["municipallity",'population growth (%)']]
df_pop_growth.head()

,municipallity,population growth (%)
0,Appingedam,-3.07
1,Delfzijl,-2.88
2,Groningen,2.44
3,Loppersum,-5.95
4,Almere,7.60


In [68]:
df_cbs = pd.merge(df_cbs, df_pop_growth)
df_cbs.head()

,municipallity,latitude,longitude,surface area (km2),population,population density per km2,approximate radius (m),house price (euro),population growth (%)
0,Aa en Hunze,53.005,6.760,279,25390,90,9423.213073,284800,0.96
1,Aalsmeer,52.267,4.750,34,31499,928,3289.549063,409300,2.52
2,Aalten,51.926,6.583,97,26962,277,5556.262774,226100,0.81
3,Achtkarspelen,53.255,6.143,104,27935,268,5753.254413,216500,-0.50
4,Alblasserdam,51.862,4.658,10,20014,2001,1784.008461,268400,1.61


## CBS data description
Above you can see a part of the dataframe which we will be using in our furter analyses. It is a dataframe containing 9 columns of relevant data which we will discuss in detail below. In total we have this information on 333 municipallities in the Netherlands. The dataframe is orded alphabetically based on the municipallity name.

##### Column 1 - municipallity
This is the name of the municipallities used by the central bureau of statistics of the Netherlands. 

##### Column 2 & 3 - Latitude and Longitude
These are the central coordinates of the different municipallities. They will be used as input for obtaining the foursquare venue data as well as plotting different maps as data visualisations.

##### column 4 - Surface area (km2)
The different municipallities have a quite a big range in de size. Bigger municipallities might have more diversification and als more available veneus. On itself a interesting parameter for further analyses and in combination with column 5 it give the avarega population density. The surface area is noted in square kilometers.

##### column 5  - population
The total number of people living in this municipallity. More people often means more money and more services. But als combined with column 4 an input for population density.

##### column 6 - population density per km2
This is the average amount of people living in one square kilometer area in this municipallity. A higher population density will be found in urban settings then in rural ones. Als an indicator for how the amount of agricultural and nature compared to housing. Also a higher population density is often combined with more available services. 

##### column 7 - approximate radius
Due to the differt sizes of the municipallities we wanted to change the radius for each API call to foursqaure to get appropriate data for each municipallity. If this will not work I will use the average of the approximate radius as input. The approximate radius is calculated by acting as if all the municipallities are perfect circles and obtaining a radius by converting the surface area of the circle to its radius. In general this should give a good approximation of the average radius of the municipallity. 

##### column 8 - house price (euro)
This is the average houseprice per municipallity as registered by the CBS. 

##### column 9 - population growth (%)
On general the population of the Netherlans is growing. This column gives an indication of how that growth is distributed throughout the municipallities. The data is based on the periode of 2015 - 2020, calculating the difference in the population in the beginning and the end of this time periode. The negative numbers are areas with population shrinkage while the positives are growth. 

## Part Two - location data using Foursquare 
The CBS data discribe in the first part of the chapter will be used to obtain data from Foursquare. First we will visualise the spatial data using a folium map to get an idea of where we are located. In a following chapter we will be using the API to collect venue data for the different municipallities. 

Foursquare is an extensive geospatial database which provide a lot of spatial data to differt companies and services. The data is continualy updated and validated by millions of consumers. For this research I am interesseted in POI's (points of interest) in the different municipallities. With the extanded database of foursqaure I will be able to obtain an large amount of relavant real life data in no time. 

According to Foursquare; 
"If it tells you where, it’s probably built on Foursquare. We believe in the power of location. What people experience in the real world and the places they go are powerful reflections of who they are and what they care about. We help leading global companies tap into this intelligence to create better customer experiences and smarter business outcomes, all based on the world’s leading platform for understanding people, places, and the interactions between them." 

In a following chapter I will use the foursquare data bases to querry for relavant venue data. This will let me explore all the nearby venues and it's characteristics. For this research I'm interested in what kind of venues are available and what will distinguish one municiapallity from another. 

The location data from foursquare will also be the input for our KNN clustering to find similar municipallies and spatial distributions of the clusters. 


In [1]:
# finding the centre of our data points to zoom into on the map
latitude_netherlands = df_cbs['latitude'].mean()
longitude_netherlands = df_cbs['longitude'].mean()

print('coordinates of the netherlands are: ', latitude_netherlands, longitude_netherlands)

NameError: name 'df_cbs' is not defined

In [2]:
# create map of Netherlans using latitude and longitude values of the municipallities to get a feel for the data

map_mun = folium.Map(location=[latitude_netherlands, longitude_netherlands], zoom_start=7)

# add markers to map
for lat, lng, mun in zip(df_cbs['latitude'], df_cbs['longitude'], df_cbs['municipallity']):
    label = '{}'.format(mun)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_mun)  
    
map_mun

NameError: name 'folium' is not defined

For every location in de Map above we will use the foursquare API to obtain data of the surrounding venues. From this venue data we will extract the venue categories, which will be stored in a dataframe.

### obtaining Foursquare location data

In [104]:
# The code was removed by Watson Studio for sharing.

In [109]:
def getNearbyVenues(names, latitudes, longitudes, radius):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['municipallity', 
                  'municipallity Latitude', 
                  'municipallity Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [110]:
Venues = getNearbyVenues(df_cbs['municipallity'], df_cbs['latitude'], df_cbs['longitude'], radius)

Aa en Hunze


KeyError: 'groups'

In [107]:
print(Venues.shape)
Venues.head(10)

(5892, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Aa en Hunze,53.005,6.76,Jumbo,53.002051,6.764763,Supermarket
1,Aa en Hunze,53.005,6.76,Jimm's,53.004209,6.763737,Restaurant
2,Aa en Hunze,53.005,6.76,Sporthal De Goorns,53.003203,6.757370,Soccer Field
3,Aa en Hunze,53.005,6.76,Actioncenter,53.002491,6.758756,Gym / Fitness Center
4,Aa en Hunze,53.005,6.76,Sportpark De Goorns,53.002376,6.757704,Soccer Field
5,Aa en Hunze,53.005,6.76,Bakkerij Job,53.002346,6.764154,Bakery
6,Aalsmeer,52.267,4.75,Café Bar Joppe,52.268649,4.748619,Bar
7,Aalsmeer,52.267,4.75,Action,52.266511,4.750560,Discount Store
8,Aalsmeer,52.267,4.75,Het Tuinhuis,52.267289,4.746148,Tea Room
9,Aalsmeer,52.267,4.75,Pasta Vino,52.268698,4.748725,Italian Restaurant


### Foursquare data description
 
Above are the first 10 entries out of the location data dataframe. Besides the municipallity data this dataframe contains the name of the venue, its coordinates and the venue categorie. The venue catergory is the thing we are going to use in further analyses. The total dataframe consists of 7 columns with 5892 venues. 

##### column 1 Municipality 
This is the name of the municipallity in which the Venue is located. We can use this column to group the data and merge it into the CBS dataset. 

##### column 2 & 3 Municipality Latitude and Longitude
Coordinates of the municipallity

##### column 4 Venue
This is the name of the Venue as it is known in the Foursquare database. This is not a unique name as there can be different branches of the same shop, like 'Jumbo' supermarket that can be located in different towns or neighborhoods. 

##### column 5 & 6 Venue Latitude and Longitude
The coordinates of the actual venue that itself 

##### column 7 Venue Category 
In the foursquare database all Venues are put in a category. This will be the main location data that we are going to use in the rest of the analyses. As this describes what kind of services and venues are available in the differtent municipalities. 



# Chapter 3 - Analyses

In [112]:
Venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Aa en Hunze,6,6,6,6,6,6
Aalsmeer,19,19,19,19,19,19
Aalten,7,7,7,7,7,7
Achtkarspelen,7,7,7,7,7,7
Alblasserdam,14,14,14,14,14,14
Albrandswaard,5,5,5,5,5,5
Alkmaar,33,33,33,33,33,33
Almelo,27,27,27,27,27,27
Almere,56,56,56,56,56,56


In [113]:
print('There are {} uniques categories.'.format(len(Venues['Venue Category'].unique())))

There are 322 uniques categories.


In [114]:
# one hot encoding
mun_onehot = pd.get_dummies(Venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
mun_onehot ['municipallity'] = Venues['Neighborhood']

# move neighborhood column to the first column
mun_onehot.set_index('municipallity', inplace=True)
mun_onehot.reset_index(inplace=True)

#display head
mun_onehot.head(12)

,municipallity,ATM,Accessories Store,Afghan Restaurant,American Restaurant,Amphitheater,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bay,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Brasserie,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Library,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cultural Center,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dutch Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Escape Room,Event Service,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fireworks Store,Fish & Chips Shop,Fish Market,Fishing Spot,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Forest,Fountain,French Restaurant,Fried Chicken Joint,Friterie,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Field,Hookah Bar,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Library,Lighthouse,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Multiplex,Museum,Music Venue,Nature Preserve,Neighborhood,New American Restaurant,Nightclub,Noodle House,Notary,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Outlet Mall,Paper / Office Supplies Store,Park,Parking,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Pet Service,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pier,Piercing Parlor,Pizza Place,Platform,Playground,Plaza,Pool,Pool Hall,Print Shop,Pub,Racetrack,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Road,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Science Museum,Seafood Restaurant,Shawarma Place,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Area,Smoke Shop,Snack Place,Soccer Field,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stables,Stadium,Steakhouse,Storage Facility,Supermarket,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Tanning Salon,Tapa

In [115]:
mun_grouped = mun_onehot.groupby('municipallity').mean()
mun_grouped.reset_index(inplace = True)

print(mun_grouped.shape)
mun_grouped.head()

(330, 323)


,municipallity,ATM,Accessories Store,Afghan Restaurant,American Restaurant,Amphitheater,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bay,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Brasserie,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Library,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cultural Center,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dutch Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Escape Room,Event Service,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fireworks Store,Fish & Chips Shop,Fish Market,Fishing Spot,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Forest,Fountain,French Restaurant,Fried Chicken Joint,Friterie,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Field,Hookah Bar,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Library,Lighthouse,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Multiplex,Museum,Music Venue,Nature Preserve,Neighborhood,New American Restaurant,Nightclub,Noodle House,Notary,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Outlet Mall,Paper / Office Supplies Store,Park,Parking,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Pet Service,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pier,Piercing Parlor,Pizza Place,Platform,Playground,Plaza,Pool,Pool Hall,Print Shop,Pub,Racetrack,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Road,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Science Museum,Seafood Restaurant,Shawarma Place,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Area,Smoke Shop,Snack Place,Soccer Field,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stables,Stadium,Steakhouse,Storage Facility,Supermarket,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Tanning Salon,Tapa

In [116]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [118]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['municipallity']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
municipallity_venues_sorted = pd.DataFrame(columns=columns)
municipallity_venues_sorted['municipallity'] = mun_grouped['municipallity']

for ind in np.arange(mun_grouped.shape[0]):
    municipallity_venues_sorted.iloc[ind, 1:] = return_most_common_venues(mun_grouped.iloc[ind, :], num_top_venues)

municipallity_venues_sorted.head(20)

,municipallity,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Aa en Hunze,Soccer Field,Bakery,Supermarket,Gym / Fitness Center,Restaurant,Zoo Exhibit,Fireworks Store,Exhibit,Falafel Restaurant,Farm
1,Aalsmeer,French Restaurant,Harbor / Marina,Supermarket,Bar,Ice Cream Shop,Restaurant,Indian Restaurant,Drugstore,Train Station,Boat or Ferry
2,Aalten,Pub,Department Store,German Restaurant,Snack Place,Discount Store,Bar,Supermarket,Flower Shop,Field,Exhibit
3,Achtkarspelen,Supermarket,Restaurant,Garden,Train Station,Drugstore,Fish & Chips Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
4,Alblasserdam,Supermarket,Restaurant,Breakfast Spot,Shopping Mall,Bakery,Drugstore,Snack Place,French Restaurant,Asian Restaurant,Outdoor Supply Store
5,Albrandswaard,Park,Supermarket,Fast Food Restaurant,Restaurant,Bus Stop,Fireworks Store,Exhibit,Falafel Restaurant,Farm,Farmers Market
6,Alkmaar,Café,Coffee Shop,Indonesian Restaurant,Comfort Food Restaurant,Bistro,Sporting Goods Shop,Cheese Shop,Church,Salad Place,Clothing Store
7,Almelo,Platform,Sandwich Place,Ice Cream Shop,Shopping Mall,Restaurant,Drugstore,Toy / Game Store,Garden Center,Bed & Breakfast,Men's Store
8,Almere,Sushi Restaurant,Clothing Store,Coffee Shop,Plaza,Bar,Bakery,Supermarket,Greek Restaurant,Restaurant,Mexican Restaurant
9,Alphen aan den Rijn,Restaurant,Japanese Restaurant,Italian Restaurant,Fast Food Restaurant,Greek Restaurant,Thai Restaurant,Drugstore,Café,Bar,Theater


In [125]:
## Merge most common venues with CBS dataframe
df_mun = df_cbs
df_mun.head()


,municipallity,latitude,longitude,surface area (km2),population,population density per km2,approximate radius (m),house price (euro),population growth (%)
0,Aa en Hunze,53.005,6.760,279,25390,90,9423.213073,284800,0.96
1,Aalsmeer,52.267,4.750,34,31499,928,3289.549063,409300,2.52
2,Aalten,51.926,6.583,97,26962,277,5556.262774,226100,0.81
3,Achtkarspelen,53.255,6.143,104,27935,268,5753.254413,216500,-0.50
4,Alblasserdam,51.862,4.658,10,20014,2001,1784.008461,268400,1.61


## KNN clustering of Venues
Based on the venue data we will run a KNN analyses to divide the data into different clusters.

In [128]:
# set number of clusters
kclusters = 5

mun_grouped_clustering = mun_grouped.drop('municipallity', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(mun_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 4, 1, 0, 0, 0, 4, 4, 4, 0], dtype=int32)

In [130]:
## add cluster labels
df_mun = pd.merge(df_mun, municipallity_venues_sorted)
df_mun['cluster'] = kmeans.labels_
df_mun.head()

,municipallity,latitude,longitude,surface area (km2),population,population density per km2,approximate radius (m),house price (euro),population growth (%),1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,cluster
0,Aa en Hunze,53.005,6.760,279,25390,90,9423.213073,284800,0.96,Soccer Field,Bakery,Supermarket,Gym / Fitness Center,Restaurant,Zoo Exhibit,Fireworks Store,Exhibit,Falafel Restaurant,Farm,0
1,Aalsmeer,52.267,4.750,34,31499,928,3289.549063,409300,2.52,French Restaurant,Harbor / Marina,Supermarket,Bar,Ice Cream Shop,Restaurant,Indian Restaurant,Drugstore,Train Station,Boat or Ferry,4
2,Aalten,51.926,6.583,97,26962,277,5556.262774,226100,0.81,Pub,Department Store,German Restaurant,Snack Place,Discount Store,Bar,Supermarket,Flower Shop,Field,Exhibit,1
3,Achtkarspelen,53.255,6.143,104,27935,268,5753.254413,216500,-0.50,Supermarket,Restaurant,Garden,Train Station,Drugstore,Fish & Chips Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,0
4,Alblasserdam,51.862,4.658,10,20014,2001,1784.008461,268400,1.61,Supermarket,Restaurant,Breakfast Spot,Shopping Mall,Bakery,Drugstore,Snack Place,French Restaurant,Asian Restaurant,Outdoor Supply Store,0


In [ ]:
# create map
map_clusters = folium.Map(location=[latitude_netherlands, longitude_netherlands], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.8).add_to(map_clusters)
       
map_clusters